In [ ]:
"""Log into nersc account and get newt session id."""

import getpass
import os
import sys

import requests

nersc_url = 'https://newt.nersc.gov/newt'


user = input('Username: ')
password = getpass.getpass(prompt='Password: ')
otp = input('Enter one time password (for MFA) ')
# print('ready')

credentials = {
    'username': user,
    'password': password + str(otp)
}

url = '{}/login/'.format(nersc_url)
r = requests.post(url, data=credentials)
r.raise_for_status()

js = r.json()
if not js.get('auth'):
    raise RuntimeError('NOT authenticated: ', r.text)

# (else)
session_id = js.get('newt_sessionid')
print('newt_sessionid: {}'.format(session_id))

output_filename = os.path.expanduser('~/.newt_sessionid')
with open(output_filename, 'w') as f:
    f.write(session_id)
    print('Wrote {}'.format(output_filename))

# Sanity check -- get user's scratch directory
cookies = dict(newt_sessionid=session_id)
data = {
    'executable': 'echo $SCRATCH',
    'loginenv': 'true'
}
machine = 'cori'
url = '%s/command/%s' % (nersc_url, machine)
r = requests.post(url, data=data, cookies=cookies)
r.raise_for_status()
print(r.json())
